In [22]:
import happybase
conn=happybase.Connection('localhost',8080)
table=conn.table('users')

In [ ]:
def get_data():
    import random
    name=input('Name: ')
    From=input('From: ')
    education=input('Education: ')
    profession=input('Profession: ')
    location=input('Current location: ')
    Birthday=input('Birthday')
    
    
    
    #id_u='R1'
    
    while True:
        num=random.randint(1, 10001)
        id_u='R'+name.replace(' ','_')+'_'+Birthday[:2]+'_'+str(num)
        #print(id_u)
    
    
        try:
            row=table.row(id_u)
            #print("b")
        except:
            row={}
        print(row)
        if len(row)==0:
            #print("c")
            break
    #id_b=''.join(format(ord(i), 'b') for i in id_u)
    #name_b=''.join(format(ord(i), 'b') for i in name)
    #from_b=''.join(format(ord(i), 'b') for i in From)
    #edu_b=''.join(format(ord(i), 'b') for i in education)
    #pro_b=''.join(format(ord(i), 'b') for i in profession)
    #loc_b=''.join(format(ord(i), 'b') for i in location)
    #bir_b=''.join(format(ord(i), 'b') for i in Birthday)
    
    
    
    #print("a")
    
    table.put(id_u, {b'cf_user_1:name': name,b'cf_user_1:from': From,b'cf_user_1:Education': education,b'cf_user_1:Profession': profession,b'cf_user_1:location': location,b'cf_user_1:birthday': Birthday}) 
    #table.put(b'R2',{b'cf_user_1:c1':b'v1',b'cf_user_1:c2':b'v2'})
        
        
        
        
    
    

In [ ]:
get_data()

In [23]:
#table.put('R3',{b'cf_user_1:c1':'v1',b'cf_user_1:c2':'v2'})
row=table.row('RAbhijit_Roy_03_2944')
print(row)

{b'cf_user_1:Education': b'NIT Durgapur', b'cf_user_1:Profession': b'Student', b'cf_user_1:birthday': b'03.10.1998', b'cf_user_1:from': b'Burdwan', b'cf_user_1:location': b'Durgapur', b'cf_user_1:name': b'Abhijit Roy'}


In [24]:
import MySQLdb
import mysql.connector
conn= mysql.connector.connect(host="localhost",user="abhijit",passwd="1234",db="users" )


In [4]:
id1='234'
email='abc@gmail.com'
passwd='1234'
mycursor = conn.cursor()
#mycursor.execute('INSERT INTO user_details (id,email,password) VALUES("%s","%s","%s")'%(id1,email,passwd))
#conn.commit()

In [31]:
from flask import Flask,request,render_template,redirect,flash
import pickle
from werkzeug import secure_filename
import numpy as np
import flask
import os


UPLOAD_FOLDER = 'uploads'

app = Flask(__name__, template_folder='templates')
#app.secret_key = "secret key"
app.config['UPLOAD_FOLDER'] = UPLOAD_FOLDER


def sendResponse(responseObj):
    response = flask.jsonify(responseObj)
    response.headers.add('Access-Control-Allow-Origin', '*')
    response.headers.add('Access-Control-Allow-Methods', 'GET')
    response.headers.add('Access-Control-Allow-Headers', 'accept,content-type,Origin,X-Requested-With,Content-Type,access_token,Accept,Authorization,source')
    response.headers.add('Access-Control-Allow-Credentials', True)
    return response

@app.route('/signup')
def signup():
    return render_template('get.html')

@app.route('/signup_details',methods=['POST'])
def signup_details():
    if request.method=='POST':
        email=request.form.get('Email')
        passwd=request.form.get('Passwd')
        con_passwd=request.form.get('Con_Passwd')
    
        if email=='' or passwd=='' or con_passwd=='':
            flash('fields empty')
            return redirect(request.url)
    
        if passwd!=con_passwd:
            flash('Password do not match')
            return redirect(request.url)
        query="""SELECT * FROM user_details WHERE email='%s'"""%(email)
        mycursor = conn.cursor()
        mycursor.execute(query)
        dup=mycursor.fetchall()
        i=mycursor.rowcount
    
        if i!=0:
            flash('Emails exist')
            return redirect(request.url)
        mycursor = conn.cursor()
        id1=email.split('@')[0]
        print(id1)
        mycursor = conn.cursor()
        mycursor.execute('INSERT INTO user_details (id,email,password) VALUES("%s","%s","%s")'%(id1,email,passwd))
        conn.commit()
        print("A")
        return render_template('signup.html',name=id1)
    else:
        return redirect(request.url)
    

@app.route('/personal_info',methods=['POST'])
def personal_info():

    fields=['usr_name','Name','From','Education','Profession','location','Birthday']
    
    record=[]
    if request.method=='POST':
        for i in range(len(fields)):
            if request.form.get(fields[i])=="":
                flash('fields missing')
                return redirect(request.url)
            record.append(request.form.get(fields[i]))
        if 'file' not in request.files:
            flash('No file part')
            return redirect(request.url)
        file = request.files['file']
        if file.filename == '':
            flash('No file selected for uploading')
            return redirect(request.url)
        if file:
            filename = secure_filename(file.filename)
            file.save(os.path.join(app.config['UPLOAD_FOLDER'],filename))
            na=os.path.join(app.config['UPLOAD_FOLDER'],filename)
        record.append(na)
        print(record)
        #print()
        while True:
            id_u='R'+record[0]
            #print(id_u)
    
    
            try:
                row=table.row(id_u)
                #print("b")
            except:
                row={}
            print(row)
            if len(row)==0:
                #print("c")
                break
        
        table.put(id_u, {b'cf_user_1:name': record[0],b'cf_user_1:from': record[1],b'cf_user_1:Education': record[2],b'cf_user_1:Profession': record[3],b'cf_user_1:location': record[4],b'cf_user_1:birthday': record[5],b'cf_user_1:profile_pic': record[6]})
        return render_template('success.html')
    else:
        return redirect(request.url)

@app.route('/login',methods=['POST'])
def login():
    if request.method=='POST':
        return render_template('login.html')
    else:
        return redirect(request.url)
@app.route('/log')
def log():
    return render_template('login.html')

@app.route('/login_det',methods=['POST'])
def login_det():
    if request.method=='POST':
        email=request.form.get('Email')
        passwd=request.form.get('Passwd')
        query="""SELECT password FROM user_details WHERE email='%s'"""%(email)
        mycursor = conn.cursor()
        mycursor.execute(query)
        
        record=mycursor.fetchall()
        
        if record==0:
            flash('The email is not registered.Please register')
            return render_template('get.html')
        
        else:
            print(record[0][0])
            if passwd==record[0][0]:
                query="""SELECT id FROM user_details WHERE email='%s'"""%(email)
                mycursor = conn.cursor()
                mycursor.execute(query)
                id1=mycursor.fetchall()
                print(id1[0][0])
                return render_template('succ_login.html',name=id1[0][0])
            else:
                flash('Wrong password. Please try again')
                return redirect(request.url)
    else:
        
        return redirect(request.url)
            
@app.route('/cont',methods=['POST'])
def cont():
    if request.method=='POST':
        id1=request.form.get('usr_name')
        return render_template('home.html',name=id1)
    else:
        return redirect(request.url)
        
        
    



    
        
if __name__ == "__main__":
    app.run()
    
    

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [24/Oct/2020 01:03:40] "POST /login_det HTTP/1.1" 200 -


1234
myac


127.0.0.1 - - [24/Oct/2020 01:03:41] "POST /cont HTTP/1.1" 200 -
